In [ ]:
import os
os.chdir("/content/drive/My Drive/Translator/")

In [ ]:
os.getcwd()

'/content/drive/My Drive/Translator'

In [ ]:
import pandas as pd
from keras import preprocessing,utils

In [ ]:
import numpy as np


In [ ]:
import re

In [ ]:
import tensorflow as tf


In [ ]:
import time


# Data Preparing

In [ ]:
# !wget http://www.manythings.org/anki/mar-eng.zip -O mar-eng.zip
# !unzip mar-eng.zip

In [ ]:
lines = pd.read_table( 'mar.txt' , names=[ 'eng' , 'mar' ] )

In [ ]:
lines.head()

,eng,mar
Go.,जा.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Run!,पळ!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
Run!,धाव!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
Run!,पळा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
Run!,धावा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [ ]:
lines.reset_index(level=0,inplace=True)

In [ ]:
lines.head()

,index,eng,mar
0,Go.,जा.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Run!,पळ!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
2,Run!,धाव!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run!,पळा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,धावा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [ ]:
lines.rename( columns={ 'index' : 'eng' , 'eng' : 'mar' , 'mar' : 'c' } , inplace=True )
lines = lines.drop( 'c' , 1 )

In [ ]:
lines.head()

,eng,mar
0,Go.,जा.
1,Run!,पळ!
2,Run!,धाव!
3,Run!,पळा!
4,Run!,धावा!


In [ ]:
lines=lines.sample(frac=1)

In [ ]:
lines.head()

,eng,mar
34962,I got the message just this morning.,मला आज सकाळीच निरोप पोहोचला.
29916,They are preparing themselves.,ते स्वतःला तयार करत आहेत.
1881,Who sent you?,तुम्हाला कोणी पाठवलं?
35270,Tom drinks six cups of coffee a day.,टॉम एका दिवसात सहा कप कॉफी पितो.
38994,"My uncle, who lives in Paris, came to see us.",माझा पॅरिसमध्ये राहणारा मामा आम्हाला भेटायला आला.


In [ ]:
len(lines)

40751

In [ ]:
train=int(0.9*40751)
val=int(0.1*40751)


In [ ]:
train

36675

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
eng_unprocessed=lines['eng'].values.tolist()

In [ ]:
mar_unprocessed=lines['mar'].values.tolist()

In [ ]:
eng_unprocessed[:4]

['I got the message just this morning.',
 'They are preparing themselves.',
 'Who sent you?',
 'Tom drinks six cups of coffee a day.']

In [ ]:
mar_unprocessed[:4]

['मला आज सकाळीच निरोप पोहोचला.',
 'ते स्वतःला तयार करत आहेत.',
 'तुम्हाला कोणी पाठवलं?',
 'टॉम एका दिवसात सहा कप कॉफी पितो.']

In [ ]:
def preprocess_sentence(w):
  
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
mar=[]
for line in mar_unprocessed:
  mar.append(preprocess_sentence(line))

In [ ]:
eng=[]
for line in eng_unprocessed:
  eng.append(preprocess_sentence(line))

In [ ]:
eng_test=eng[:val]

In [ ]:
mar_test=mar[:val]

In [ ]:
eng=eng[val:]

In [ ]:
mar=mar[val:]

In [ ]:
len(eng)

36676

# DATA GENERATOR

## Encoder Input

In [ ]:
eng_lines=[]
for line in eng:
  eng_lines.append(line)

In [ ]:
eng_lines[0]

'<start> I still remember Tom . <end>'

In [ ]:
eng_tokenizer=preprocessing.text.Tokenizer(oov_token='UNK',filters='')
eng_tokenizer.fit_on_texts(eng_lines)
token_eng_lines=eng_tokenizer.texts_to_sequences([eng_lines[0]])

In [ ]:
token_eng_lines

[[2, 6, 92, 318, 7, 4, 3]]

In [ ]:
eng_tokenizer.sequences_to_texts(token_eng_lines)

['<start> i still remember tom . <end>']

In [ ]:
length_list=list()
for line in eng_lines:
  
  length_list.append(len(line.split()))

max_eng_length=int(np.percentile(length_list,100))

In [ ]:
max_eng_length

42

In [ ]:
eng_tokenizer.word_index['PAD0']=0

In [ ]:
eng_ix_to_word = {}

#no word in vocab has index 0, but padding will have index 0
eng_tokenizer.word_index['PAD0'] = 0 
for word in eng_tokenizer.word_index:
  eng_ix_to_word[eng_tokenizer.word_index[word]] = word


In [ ]:
eng_word_dict=eng_tokenizer.word_index
num_eng_tokens=len(eng_word_dict)+1


In [ ]:
eng_ix_to_word = {}

#no word in vocab has index 0, but padding will have index 0
for word in eng_tokenizer.word_index:
  eng_ix_to_word[eng_tokenizer.word_index[word]] = word


In [ ]:
len(eng_ix_to_word)==len(eng_word_dict)

True

In [ ]:
eng_tokenizer.index_word[1]

'UNK'

In [ ]:
num_eng_tokens

5739

In [ ]:
eng_lines=eng_tokenizer.texts_to_sequences(eng_lines)

In [ ]:
eng_lines=preprocessing.sequence.pad_sequences(eng_lines,maxlen=max_eng_length,padding='post')

In [ ]:
eng_lines.shape

(36676, 42)

## Decoder input

In [ ]:

mar_lines=[]
for line in mar:
  mar_lines.append(line)

In [ ]:
mar_lines[0]

'<start> मला अजूनही टॉम आठवतो . <end>'

In [ ]:

mar_tokenizer=preprocessing.text.Tokenizer(oov_token='UNK',filters='')
mar_tokenizer.fit_on_texts(mar_lines)
token_mar_lines=mar_tokenizer.texts_to_sequences([mar_lines[0]])

In [ ]:
token_mar_lines

[[2, 9, 76, 10, 2132, 4, 3]]

In [ ]:
mar_tokenizer.sequences_to_texts(token_mar_lines)

['<start> मला अजूनही टॉम आठवतो . <end>']

In [ ]:
length_list=list()
for line in mar_lines:
  
  length_list.append(len(line.split()))

max_mar_length=int(np.percentile(length_list,100))

In [ ]:
max_mar_length

40

In [ ]:
mar_word_dict=mar_tokenizer.word_index
num_mar_tokens=len(mar_word_dict)+1

In [ ]:
mar_ix_to_word = {}

#no word in vocab has index 0, but padding will have index 0
for word in mar_tokenizer.word_index:
  mar_ix_to_word[mar_tokenizer.word_index[word]] = word

In [ ]:
len(mar_ix_to_word)==len(mar_word_dict)

True

In [ ]:
num_mar_tokens

13261

In [ ]:
mar_lines=mar_tokenizer.texts_to_sequences(mar_lines)

In [ ]:
mar_lines=preprocessing.sequence.pad_sequences(mar_lines,maxlen=max_mar_length,padding='post')

In [ ]:
mar_lines.shape

(36676, 40)

Generator

In [ ]:
BUFFER_SIZE = len(mar_lines)
BATCH_SIZE = 64
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = num_eng_tokens
vocab_tar_size = num_mar_tokens

dataset = tf.data.Dataset.from_tensor_slices((eng_lines,mar_lines)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 42]), TensorShape([64, 40]))

In [ ]:
example_target_batch

<tf.Tensor: shape=(64, 40), dtype=int32, numpy=
array([[   2,   29, 3206, ...,    0,    0,    0],
       [   2,  328,  140, ...,    0,    0,    0],
       [   2,    7, 5998, ...,    0,    0,    0],
       ...,
       [   2,    7,  160, ...,    0,    0,    0],
       [   2,  145, 3032, ...,    0,    0,    0],
       [   2,   30,  120, ...,    0,    0,    0]], dtype=int32)>

In [ ]:
eng_ix_to_word[0]

'PAD0'

# Model

encoder

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))


In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 42, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 42, 1)


In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 13261)


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

# Train

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
def evaluate(sentence):
  

  inputs = [eng_word_dict.get(i,1) for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_eng_length,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([mar_tokenizer.word_index['<start>']], 0)

  for t in range(max_mar_length):
    predictions, dec_hidden, _ = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += mar_tokenizer.index_word[predicted_id] + ' '

    if mar_tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [ ]:
r,s=evaluate('what happened to you')

In [ ]:
type(s)

str

In [ ]:
def evaluate_model(inputs,targets):
  actual,predicted=[],[]
  i=1
  for input,target in zip(inputs,targets):
    i+=1
    yhat,_=evaluate(input)

    actual.append([target.split()])
    predicted.append(yhat.split())
    if i%1000==0:
      print(i,'sentences processed')
  b1 = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
  b2 = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
  b3 = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0))
  b4 = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))
  print('\n')
  #score for 1-gram model
  print('BLEU-1: %f' % b1)
  #50% score calculated on 1-gram and other 50% using 2-gram
  print('BLEU-2: %f' % b2)
  #33% to each 1,2 and 3 gram
  print('BLEU-3: %f' % b3)
  # 1/4 score of each gram
  print('BLEU-4: %f' % b4)
  return [b1,b2,b3,b4]

  
    



In [ ]:
from nltk.translate.bleu_score import corpus_bleu
reference = [[['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']],[['I','AM',"good"]]]
candidate = [['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog'],['I','AM',"good"]]
score = corpus_bleu(reference, candidate)
print(score)

0.9621954581957615


In [ ]:
eng_test[:10]

['<start> I got the message just this morning . <end>',
 '<start> They are preparing themselves . <end>',
 '<start> Who sent you ? <end>',
 '<start> Tom drinks six cups of coffee a day . <end>',
 '<start> My uncle , who lives in Paris , came to see us . <end>',
 '<start> Thank you for today . <end>',
 "<start> I'm thinking of changing jobs . <end>",
 '<start> Whose food is this ? <end>',
 '<start> No one has been arrested . <end>',
 '<start> My brother uses it . <end>']

In [ ]:
mar_test[:10]

['<start> मला आज सकाळीच निरोप पोहोचला . <end>',
 '<start> ते स्वतःला तयार करत आहेत . <end>',
 '<start> तुम्हाला कोणी पाठवलं ? <end>',
 '<start> टॉम एका दिवसात सहा कप कॉफी पितो . <end>',
 '<start> माझा पॅरिसमध्ये राहणारा मामा आम्हाला भेटायला आला . <end>',
 '<start> आजबाबत धन्यवाद . <end>',
 '<start> मी नोकरी बदलायचा विचार करत आहे . <end>',
 '<start> हे कोणाचं जेवण आहे ? <end>',
 '<start> कोणालाही अटक झाली नाही आहे . <end>',
 '<start> माझा भाऊ वापरतो . <end>']

In [ ]:
evaluate_model(eng_test[:10],mar_test[:10])



BLEU-1: 0.000000
BLEU-2: 0.000000
BLEU-3: 0.000000
BLEU-4: 0.000000


[0, 0, 0, 0]

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0
  
  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([mar_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 16
history={'loss':[], 'BLEU_val':[]}
for epoch in range(EPOCHS):
  start = time.time()
  
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('\nEvaluating model for {} epoch '.format(epoch+1))
  history['loss'].append(total_loss/steps_per_epoch)
  history['BLEU_val'].append(evaluate_model(eng_test,mar_test))

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.6946
Epoch 1 Batch 100 Loss 0.9395
Epoch 1 Batch 200 Loss 0.7443
Epoch 1 Batch 300 Loss 0.7497
Epoch 1 Batch 400 Loss 0.7751
Epoch 1 Batch 500 Loss 0.7265

Evaluating model for 1 epoch 
1000 sentences processed
2000 sentences processed
3000 sentences processed
4000 sentences processed


BLEU-1: 0.273859
BLEU-2: 0.200534
BLEU-3: 0.132107
BLEU-4: 0.042951
Epoch 1 Loss 0.8343
Time taken for 1 epoch 476.59477496147156 sec

Epoch 2 Batch 0 Loss 0.6969
Epoch 2 Batch 100 Loss 0.7062
Epoch 2 Batch 200 Loss 0.6711
Epoch 2 Batch 300 Loss 0.6784
Epoch 2 Batch 400 Loss 0.6938
Epoch 2 Batch 500 Loss 0.6259

Evaluating model for 2 epoch 
1000 sentences processed
2000 sentences processed
3000 sentences processed
4000 sentences processed


BLEU-1: 0.251901
BLEU-2: 0.189178
BLEU-3: 0.124226
BLEU-4: 0.027861
Epoch 2 Loss 0.6868
Time taken for 1 epoch 420.51189661026 sec

Epoch 3 Batch 0 Loss 0.6596
Epoch 3 Batch 100 Loss 0.5604
Epoch 3 Batch 200 Loss 0.5802
Epoch 3 Batch 300 Loss 

# Test

In [ ]:
def evaluate(sentence):

  sentence=preprocess_sentence(sentence)
  

  inputs = [eng_word_dict.get(i,1) for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_eng_length,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([mar_tokenizer.word_index['<start>']], 0)

  for t in range(max_mar_length):
    predictions, dec_hidden, _ = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += mar_tokenizer.index_word[predicted_id] + ' '

    if mar_tokenizer.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [ ]:
r,s=evaluate('what is your ')

In [ ]:
r

'तुझं नाव काय ? <end> '